# Brainstorm Proyecto 2 - Grafos

El primer paso es definir las entradas y salidas del problema

| Entrada / Salida | Nombre | Tipo | Descripción |
|-|-|-|-|
| E | C | **List** of **Celulas** | El conjunto de celulas donde encontramos su identificador, su posición, su tipo y sus péptidos asociados |
| E | d | **nat** | Distancia máxima entre células que se pueden enviar mensajes |
| s | c | **nat** | Identificador de la célula calculadora que se va a  |
| s | M | **nat** | Cantidad máxima de mensajes que se pueden enviar sin quitar células |
| s | m | **nat** | Cantidad máxima de mensajes que se pueden enviar quitando la célula calculadora indicada |

A partir del esquema mencionado en el enunciado, se tiene un problema  de flujo. Esto por lo siguiente:

1. La cantidad de mensajes recibidos y emitidos por las celulas calculadoras deben ser idénticos (Se de cumplir el principio de conservación).
2. Las celulas iniciadores reciben impulsos externos y envian mensajes a las celulas calculadoras (Son las fuentes de mensajes)
3. Las celulas ejecutadores reciben los mensajes de las celulas alculadores y ejecutan acciones en base a eso (Son los destinos de los mensajes).
4. La cantidad máxima de mensajes que se pueden enviar entre celulas esta definido por la cantidad de péptidos compartidos (Existe una capacidad máxima de mensajes que se pueden enviar entre celulas calculadoras)

Dado esto, podemos definir que la siguiente idea de algoritmo para seguir:

```
    1. Inicializar grafo desde entrada
    2. Calcular flujo máximo sobre el grafo
    3. Para cada nodo de celula calculadora del grafo:
    4.      Calcular flujo maximo sin la celula marcada
    5.      Verificar si es menor a los flujos ya obtenidos (De ser asi, se guarda el flujo y la celula)
    6. Retornar flujo maximo, celula obtenida y flujo excluyendo dicha celula
```



En ese sentido, la complejidad del algoritmo estará fuertemente influenciada por que estrategia se use para resolver el problema de máximo flujo

Esta estrategia, directamente va a explorar que sucede cuando excluimos todos los nodos calculadores. Podria entenderse que es una estrategia de busqueda completa; por lo que explora todas las posibles exclusiones de nodos y busca a partir de esto el nodo especifico que minimiza el paso de flujo

Sin embargo, podemos definir una propiedad que es el flujo que pasa a través de un nodo.

A partir de esto, tendremos en cuenta el siguiente concepto:

* **Flujo del nodo** Se entiende como el flujo que pasa a través del nodo. En ese sentido, este valor se puede hallar sumando todos los flujos que tengan de destino el nodo de interés o sumando todos los flujos que parten a partir del nodo (Por las propiedades de un flujo, sabemos que ambos valores son equivalentes)

Bajo este concepto, se tiene de idea tentativa de que el nodo que se debe excluir para minimizar el flujo total, es aquel cuyo flujo individual sea maximo (Suena enredado, pero se podria probar).

Si esta propiedad es correcta, se tendría el siguiente algoritmo tentativo.


```
    1. Inicializar grafo desde entrada
    2. Calcular flujo máximo sobre el grafo
    3. Identificar el flujo de nodo máximo y su nodo calculador asociado
    4. Recalcular el flujo excluyendo el nodo identificado
    5. Retornar flujo maximo, celula identificada y flujo excluyendo dicha celula
```

In [24]:
!java ProblemaP2.java < P1.in

4 3 1
5 7 4
3 2 1


In [25]:
!java ProblemaP2BruteForce.java < P1.in

4 3 1
5 7 4
3 2 1


In [26]:
%%timeit
!java ProblemaP2.java < P1.in > P1.out

1.24 s ± 29.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [27]:
%%timeit
!java ProblemaP2BruteForce.java < P1.in > P1.out

1.21 s ± 45.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [21]:
import random
import string

def compareResults(r1,r2):
    r1 = r1.split(' ')
    r2 = r2.split(' ')
    return r1[1]==r2[1] and r1[2]==r2[2]

def generate_random_strings(n, length=5):
    characters = string.ascii_letters + string.digits
    return [''.join(random.choice(characters) for _ in range(length)) for _ in range(n)]

def randomCaseGenerator(numNodes = 3, aminoacids = 1, maxL = 20):
    case = '1\n'
    d = random.randint(1, maxL//2)
    amino = generate_random_strings(aminoacids)
    case += f'{numNodes} {d}\n'

    t = 1
    init_amino_set = set()
    n = random.randint(1, numNodes-2)

    for i in range(1,n+1):
        x = random.randint(1, 10)
        y = random.randint(1, 10)
        aminoSize = random.randint(1, aminoacids)
        aminoSet = random.sample(amino, aminoSize)
        init_amino_set.update(aminoSet)
        aminoSet = ' '.join(aminoSet)
        case += f'{i} {x} {y} {t} {aminoSet}\n'

    t = 2
    calc_amino_set = set()
    n1 = random.randint(1, numNodes-n-1)

    for i in range(n+1,n+n1+1):
        x = random.randint(1, 10)
        y = random.randint(1, 10)
        aminoSize = random.randint(1, aminoacids)
        aminoSet = set()
        aminoSet.update(random.sample(amino, aminoSize))
        aminoSize = random.randint(1, len(init_amino_set))
        aminoSet.update(random.sample(list(init_amino_set), aminoSize))
        calc_amino_set.update(aminoSet)
        aminoSet = ' '.join(aminoSet)
        case += f'{i} {x} {y} {t} {aminoSet}\n'

    t = 3

    for i in range(n+n1+1, numNodes + 1):
        x = random.randint(1, 10)
        y = random.randint(1, 10)
        aminoSize = random.randint(1, aminoacids)
        aminoSet = set()
        aminoSet.update(random.sample(amino, aminoSize))
        aminoSize = random.randint(1, len(calc_amino_set))
        aminoSet.update(random.sample(list(calc_amino_set), aminoSize))
        aminoSet = ' '.join(aminoSet)
        if i<numNodes : case += f'{i} {x} {y} {t} {aminoSet}\n'
        else: case += f'{i} {x} {y} {t} {aminoSet}'
    return case

In [8]:
print(randomCaseGenerator(numNodes = 7, aminoacids = 5, maxL = 20))

1
7 9
1 6 10 1 liByS qeAl5 bbp8p YqEjT
2 5 1 1 ia5DA YqEjT qeAl5
3 5 5 1 bbp8p liByS YqEjT ia5DA
4 9 7 2 liByS ia5DA bbp8p
5 9 10 2 YqEjT ia5DA
6 9 10 3 YqEjT liByS ia5DA bbp8p
7 9 10 3 YqEjT ia5DA liByS qeAl5 bbp8p


In [76]:
import subprocess
import random

n = 10
cases = []
for i in range(n):
    numNodes = random.randint(3,10)
    aminoacids = random.randint(1,10)
    case = randomCaseGenerator(numNodes, aminoacids)
    cases.append(case)
    result = subprocess.run(['java', 'ProblemaP2.java'], input=case, text=True, capture_output=True)
    if type(result.stdout) != str : print('Error')
    else: print(f'Case {i} finished with n={numNodes} and m={aminoacids}\nAnswer: {result.stdout}')


Case 0 finished with n=10 and m=1
Answer: 9 1 0

Case 1 finished with n=8 and m=4
Answer: 4 6 4

Case 2 finished with n=5 and m=5
Answer: 4 0 0

Case 3 finished with n=10 and m=2
Answer: 5 0 0

Case 4 finished with n=7 and m=1
Answer: 5 0 0

Case 5 finished with n=6 and m=10
Answer: 5 3 0

Case 6 finished with n=5 and m=8
Answer: 4 5 0

Case 7 finished with n=10 and m=9
Answer: 5 44 28

Case 8 finished with n=10 and m=4
Answer: 5 3 1

Case 9 finished with n=6 and m=8
Answer: 5 0 0



Podemos hacer la comparativa sencilla de 10 casos aleatorios

In [93]:
import subprocess
import random

n = 10
cases = []
for i in range(n):
    numNodes = random.randint(3,10)
    aminoacids = random.randint(1,10)
    case = randomCaseGenerator(numNodes, aminoacids)
    result1 = subprocess.run(['java', 'ProblemaP2.java'], input=case, text=True, capture_output=True)
    result2 = subprocess.run(['java', 'ProblemaP2BruteForce.java'], input=case, text=True, capture_output=True)
    if type(result1.stdout) != str or type(result2.stdout) != str: print('Error')
    else: 
        print(f'Case {i} finished with n={numNodes} and m={aminoacids}')
        if not compareResults(result1.stdout,result2.stdout):
            cases.append(case)

print(f'Number of failed cases: {len(cases)}')

Case 0 finished with n=8 and m=4
Case 1 finished with n=3 and m=7
Case 2 finished with n=5 and m=4
Case 3 finished with n=3 and m=4
Case 4 finished with n=9 and m=7
Case 5 finished with n=6 and m=5
Case 6 finished with n=9 and m=9
Case 7 finished with n=8 and m=4
Case 8 finished with n=4 and m=10
Case 9 finished with n=3 and m=4
Number of failed cases: 0


Se realiza ahora una busqueda exhaustiva sobre 100 casos que pueden ser un poco mas grandes

In [ ]:
import subprocess
import random
import time

n = 500
cases = []
start = time.time()
for i in range(n):
    numNodes = random.randint(3,6)
    aminoacids = random.randint(1,10)
    case = randomCaseGenerator(numNodes, aminoacids)
    result1 = subprocess.run(['java', 'ProblemaP2.java'], input=case, text=True, capture_output=True)
    result2 = subprocess.run(['java', 'ProblemaP2BruteForce.java'], input=case, text=True, capture_output=True)
    if type(result1.stdout) != str or type(result2.stdout) != str: print('Error')
    else: 
        if not compareResults(result1.stdout,result2.stdout):
            print(f'Case failed after {i+1} random tests')
            cases.append(case)
            break
end = time.time()
if len(cases)==0: print(f'{n} cases were done successfully')


elapsed_time = end - start
hours = int(elapsed_time // 3600)
minutes = int((elapsed_time % 3600) // 60)
seconds = int(elapsed_time % 60)
milliseconds = int((elapsed_time * 1000) % 1000)

print(f"Execution time: {hours}h {minutes}m {seconds}s {milliseconds}ms")

Case failed after 1 random tests
Execution time: 0h 0m 1s 899ms


In [61]:
print(cases[0])

1
4 9
1 7 5 1 3bgnN TCHRd Ib4ET qp1wO
2 10 4 1 PrNUB BMxyZ 3bgnN ymUmP 7R5uC k7X93 6eNJa TCHRd gxC3v FYfmT qp1wO gSVvj Ib4ET
3 2 5 2 ymUmP TCHRd k7X93 PrNUB 3bgnN 6eNJa FYfmT BMxyZ
4 8 6 3 ymUmP Ib4ET 7R5uC gSVvj k7X93 qp1wO 3bgnN 6eNJa FYfmT BMxyZ


In [22]:
!java ProblemaP2.java < P1_test.in

3 17 7


In [23]:
!java ProblemaP2BruteForce.java < P1_test.in

3 17 7


In [5]:
!java ProblemaP2.java < P1_test_curious.in

^C
